# TextRank
- 키워드 추출
    - 단어(마디, node) 그래프를 만들어야 함
- 핵심 문장 추출

## 0. 데이터 불러오기

In [2]:
# 데이터 불러오기
import pandas as pd
data = pd.read_csv('../../data/재보궐선거댓글데이터_최종_유튜브수정_0429.csv')
# 댓글만 가져오기
text = data['댓글']

# 데이터 타입 str으로 바꿔주기
sentences = text.apply(lambda x: str(x)).to_list()[:100]

## 1. 키워드 추출

### 1.1 키워드 추출을 위해 단어 그래프 만드는 함수
- scan_vocabulary 함수 인자
    - `min_count` : 주어진 문서 집합에서 최소 빈도수 이상 등장하는 단어
    - `sents` : list of str 형식의 문장
    - `tokenize` : str형식의 문장을 list of str형식의 단어열로 나누는 토크나이저

In [2]:
from collections import Counter

def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w: c for w, c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab: idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

- idx_to_vocab : 토큰화 한 후 단어의 수를 내림차순으로 정렬한 리스트 
- vocab_to_idx : 토큰화 한 후 단어의 수를 내림차순으로 정렬한 후의 인덱스 값을 부여한 set

### 1.2 단어간 유사도 측정 함수
- Co-occurrence는 문장 내에서 두 단어의 간격이 `window` 인 횟수

In [3]:
from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k: v for k, v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

### 1.3 dictionary를 sparse matrix로 변환하는 함수

In [4]:
from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

### 1.4 단어 그래프 만드는 함수

In [5]:
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence)
    return g, idx_to_vocab

### 1.5 PageRank를 학습하는 함수 만들기

In [6]:
import numpy as np
from sklearn.preprocessing import normalize


def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    # column sum이 1이 되도록 L1 normalization 사용 = A
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1, 1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1, 1)

    # iteration
    for _ in range(max_iter):
        # A*B = column j에서 row j로의 랭킹 Rj의 전달되는 값
        R = df * (A * R) + bias

    return R

### 1.6 키워드 추출 함수

In [7]:
def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(
        sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

## 2. 문장 그래프 만들기
- TextRank를 이용해 핵심 문장을 추출하기 위해선 문장 그래프를 만들어야 함. 
- **이때 마디(node)는 각 문장이 마디가 된다.**

### 2.1 문장간 유사도 측정 함수 
- Q. 문장간 유사도 측정은 어떻게? 
- A. 두 문장에 공통으로 등장한 단어의 개수를 각 문장의 단어 개수의 log 값의 합으로 나눈 값을 유사도로 설정   
    - 문장 길이에 log를 부여하기 때문에 **길이가 길어질수록 분모 값의 증가분은 줄어든다.** 대신 길이가 길기 때문에 **다른 문장과 중복된 단어가 등장할 가능성은 높아짐**


- (1) 문장 그래프 형성 함수
    - `min_sim` : 문장 간 그래프의 sparsity가 클수록 PageRank의 계산이 빠름. 문장 간 유사도가 0.3보다 작을 경우 edge를 연결하지 않음
- (2) TextRank 문장 간 유사도 측정 함수
- (3) Cosine 유사도 측정 함수


In [8]:
from collections import Counter
from scipy.sparse import csr_matrix
import math

def sent_graph(sents, tokenize, similarity, min_count=2, min_sim=0.3):
    _, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)

    tokens = [[w for w in tokenize(sent) if w in vocab_to_idx] for sent in sents]
    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        for j, tokens_j in enumerate(tokens):
            if i >= j:
                continue
            sim = similarity(tokens_i, tokens_j)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))

def textrank_sent_sim(s1, s2):
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common / base

def cosine_sent_sim(s1, s2):
    if (not s1) or (not s2):
        return 0

    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)
    return prod / (norm1 * norm2)

In [9]:
textrank_sent_sim( sentences[1], sentences[2])

0.804112726507858

### 2.2 핵심 문장 추출 함수

In [11]:
def textrank_keysentence(sents, tokenize, min_count, similarity, df=0.85, max_iter=30, topk=5):
    g = sent_graph(sents, tokenize, min_count, similarity)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keysents = [(idx, R[idx], sents[idx]) for idx in reversed(idxs)]
    return keysents

# textrank 라이브러리 사용

In [3]:
import pandas as pd
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from textrank import extractSentences, extractKeyphrases, buildGraph

### Error 처리
- textrank를 실행할 때 nltk 라이브러리의 tagger를 다운받으라는 오류
    - `>>> import nltk`
    - `>>> nltk.download('averaged_perceptron_tagger')`

### Error처리 (2)
`java.lang.VirtualMachineErrorPyRaisable: java.lang.OutOfMemoryError: GC overhead limit exceeded`

- `>>> konlpy.jvm.init_jvm(jvmpath=None, max_heap_size=1024)`
- 2번째 인자인 max_heap_size를 증가시키면 out of memory 현상이 좀 완화되는 것

### tokenizer
- Komoran 사용
    - 명사, 동사, 형용사, 어간의 품사만 이용하는 토크나이저

In [4]:
import konlpy
from konlpy.tag import Kkma

konlpy.jvm.init_jvm(jvmpath=None, max_heap_size=1024)

In [5]:
komoran = Komoran()
# tagger = Kkma()

def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if (
        '/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words


# 토크나이저를 댓글에 적용
words = komoran_tokenize(' '.join(sentences))
# words

# 리스트 내의 문자열을 하나의 문자열로 통합
texts = ' '.join(sentences)

In [6]:
new_sentences = []
for i in sentences:
    new_sentences.append(' '.join(i.split('.')))

### textrank 사용

In [7]:
# (1) git으로 받은 모델
from textrank_files import KeywordSummarizer, KeysentenceSummarizer

keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)

keywords = keyword_extractor.summarize(sentences, topk=30)
keywords

[('하/VV', 12.26218519084091),
 ('것/NNB', 11.829102687851451),
 ('되/VV', 6.283138400653213),
 ('국민/NNG', 5.891762769295998),
 ('없/VA', 5.0661024232458995),
 ('수/NNB', 4.618087666331497),
 ('있/VV', 4.371187083216668),
 ('선거/NNG', 4.108651372311032),
 ('문재인/NNP', 3.737666963904774),
 ('정치/NNG', 3.5861844560049607),
 ('안철수/NNP', 3.44862196562734),
 ('시장/NNG', 3.3976128486458106),
 ('서울/NNP', 3.3137823080666378),
 ('시장/NNP', 3.237090571839366),
 ('정권/NNG', 3.172585809705201),
 ('총장/NNP', 3.1705407343498013),
 ('대통령/NNG', 3.0001351158327982),
 ('윤석열/NNP', 2.9638625707083537),
 ('보/VV', 2.8828240913216487),
 ('시민/NNG', 2.7825171005872957),
 ('때/NNG', 2.71804333779357),
 ('이번/NNG', 2.7027652211148725),
 ('만들/VV', 2.6971380541041867),
 ('윤/NNP', 2.643281813253993),
 ('검찰/NNG', 2.6121447665127193),
 ('정책/NNG', 2.585480911825317),
 ('정권/NNP', 2.415209713311856),
 ('민주당/NNP', 2.3672265578805307),
 ('후보/NNG', 2.310579294148303),
 ('가덕도/NNP', 2.269916625887083)]

In [8]:
# (2) textrank 모듈의 extractKeyphrases 모델
extractKeyphrases( texts )

{"'부패완판",
 'OOO들',
 'OOO정권은',
 'WAY로',
 '가능성이',
 '가덕도에 퍼붓겠다는',
 '가덕도의 겨울바닷물이',
 '가독도신공항은 오거돈재산',
 '가졌었는데',
 '간잽이인',
 '갈것이요',
 '감옥으로',
 '강도높게',
 '강아지가',
 '같습니다',
 '개소리다',
 '개표에서',
 '개헌론자인',
 '개헌운운이냐',
 '개헌하겠다고',
 '개헌하려면 임기초에',
 '개헌하지말고 국민투표로',
 '거둘것이다',
 '거릴때부터',
 '건설교통',
 '걸림돌이 없어지므로서',
 '검찰개혁의',
 '검찰총장',
 '검찰총장으로서의',
 '검찰총장을 정치적으로',
 '검찰총장직을 벗어던지고',
 '것입니다',
 '겨울바닷물이 펄펄끓는꿈을',
 '겪어본사람들의',
 '결정된다는',
 '결탁하여',
 '경고한다',
 '경선으로',
 '계류중인 울산시장',
 '고스란히 시민들에게',
 '고추가루를 뿌리고있다',
 '공산국가처럼',
 '공수처에 수사의뢰하고',
 '과감하게',
 '과학기술',
 '관철시키고 현정부여당의',
 '교수처럼',
 '교체하는게 바람직하다고',
 '구세주가 나타날때도',
 '구세주가 되어주시길',
 '국가발전을',
 '국가원수급 환영인파에',
 '국가전복',
 '국무총리',
 '국민들도',
 '국민들에게',
 '국민들의',
 '국민들한테',
 '국민앞에 타락한권력으로',
 '국민에게',
 '국민을위해',
 '국민의당에서',
 '국민의힘은',
 '국민통합 좋아하네',
 '국민투표로 국민동의를',
 '국민혈세 쳐부어서라도',
 '국정농단 범죄행위에',
 '국정농단행위를 철저하게',
 '국호의장',
 '국회의원 만들어서는',
 '국회의원3명뿐이다',
 '국회의원선거도 다시해야한다',
 '굳히기에 나설건가',
 '권력있다 부각시켜',
 '권력층의',
 '그누구도 이번보선에',
 '그동안의 행적으로',
 '그렇다고 성추행사건으로',
 '그만둬도 그만둬야할',
 '그자리까지 올라갔으면',
 '급선무다',
 '기본권을 침해하는

In [9]:
# (1) git으로 받은 모델
summarizer = KeysentenceSummarizer(tokenize = komoran_tokenize, min_sim = 0.3)
keysents = summarizer.summarize(sentences, topk=1)
keysents

[(74,
  2.517619670251482,
  '현 윤총장을 정치화한것은 정부여당이다. 무리한 현행법위반을 밥먹듯 독재,편협,지독한 편향적 논리에 매몰되어 자업자득이 된것이다. 지리멸렬한 야당에게는 어부지리로 호재가 되었다. 지각한자들은 검찰개혁의 오래된 숙원의 본질을 관철시키고 현정부여당의 무리한 불법,편법,비정상적정책 ex)가덕도,원자로폐쇄,라임,울산시장선거개입,조국가족비리,추미애법무부사유화와농락 ....etc 헤아릴수없는 파쇼독재정권의 비리들을 수사하여 불공정팽창과 기회균등을 파괴한 현정부,여당의 심판을 준비 해야 한다는 것이다. 반면교사삼아 좌,우대립에 열일하는 정치인들의 경종이 되어야 한다. 국가 발전과 국민들의 기초적 의,식,주,와 기본적인 행복권을 위한 일에 좌,우 극단적 대립들이 설자리가 아니다 그들의 밥그릇만 공공히될뿐이다. 선의의 대의정치,행정에 이념의 철지난 장난감으로 혼한을 생산하는 정치아류들의 설자리을 없애야 한다. 우리는 일류 대한민국 국민들이다. 삼류정치이단의 무리배들에게 엄중히 경고한다.')]

In [10]:
## (1) 문장을 끊고 다시 해본 textrank
sentences


summarizer = KeysentenceSummarizer(tokenize = komoran_tokenize, min_sim = 0.3)
keysents = summarizer.summarize(new_sentences, topk=1)
keysents

[(74,
  2.4144801273723524,
  '현 윤총장을 정치화한것은 정부여당이다  무리한 현행법위반을 밥먹듯 독재,편협,지독한 편향적 논리에 매몰되어 자업자득이 된것이다  지리멸렬한 야당에게는 어부지리로 호재가 되었다  지각한자들은 검찰개혁의 오래된 숙원의 본질을 관철시키고 현정부여당의 무리한 불법,편법,비정상적정책 ex)가덕도,원자로폐쇄,라임,울산시장선거개입,조국가족비리,추미애법무부사유화와농락     etc 헤아릴수없는 파쇼독재정권의 비리들을 수사하여 불공정팽창과 기회균등을 파괴한 현정부,여당의 심판을 준비 해야 한다는 것이다  반면교사삼아 좌,우대립에 열일하는 정치인들의 경종이 되어야 한다  국가 발전과 국민들의 기초적 의,식,주,와 기본적인 행복권을 위한 일에 좌,우 극단적 대립들이 설자리가 아니다 그들의 밥그릇만 공공히될뿐이다  선의의 대의정치,행정에 이념의 철지난 장난감으로 혼한을 생산하는 정치아류들의 설자리을 없애야 한다  우리는 일류 대한민국 국민들이다  삼류정치이단의 무리배들에게 엄중히 경고한다 ')]

In [17]:
# (2) 핵심 문장 추출하기
key_sentences = extractSentences( texts ) # expected string or bytes-like object => 문자열 넣기
key_sentences

'OOO들 언젠가 다 동부구치소로 갈날이 올거다 영양가도없는 헛수고 많이하네 차라리 가덕도의 겨울바닷물이 펄펄끓는꿈을 꾸는게 낫겠다 김형오가 좌파인 문재인 정부의 무소불위 권력남용을 마지막 대통령에 님짜가지 붙인 조선의 어느 왕보다 더 권력있다 부각시켜 비아냥 거릴때부터 좌파로 인한 국가 위기를 자신들의 권력 강화인 이기적 내각제로 틀어버릴 재료로 삼는 이땅의 보수라는 정당에 기생하며 순색 아닌 잡색으로 내내 내려오는 부산의 김무성, 김형오, 유승민, 박형준 등이 광주의 민주화 파는 5.18 국가전복 혁명 세력과 결탁하여 마치 대통령제 자체가 문제인것처럼 저들이 탄핵시킨 박근혜 전 대통령과 문재인을 한대 묶어 권력 남용의 대통령으로 몰아 합리화 시키려는 내각제라는 완성된 시나리오로 종결 전도시킴으로서 또 다시 국민을 현혹하는 포퓰리즘으로 국회를 입법자 마음대로의 법률과 국민의'

In [25]:
## (2) 문장을 끊고 다시 해본 extractSentences( texts )
new_texts = ' '.join(new_sentences)
extractSentences( new_texts )

'반성해야하고 국민을 대표해서 인물을 키워야 한다 문재인과 비교해 봐라 머저리 멍청이와 용감한 남자 서울대 법대를 나온 훌륭한 대한민국인재다 나라의 복이다 젱세균!글자 그대로 세균만한 인간이 x 새끼마냥 찍찍거리는 꼴이 참 가관 윤 총장이 너같은 x 새끼하고 놀 존재인가?너 꼬라지를 알아라 검찰총장직으로는 문재인정권의 부도덕한 조직적 꼼수 국정농단 범죄행위에 대해서 정의롭게 수사하고 처벌하는 수단이 이미 문재인에 의해서 차단되어 불가하니 ~ 이제는 검찰총장직을 벗어던지고 차기대통령으로 정정당당하게 선출되어 문재인정권의 헌법파괴 사법농단 국정농단행위를 철저하게 파해치고 모두 감옥으로 보내어 피눈물을 흘리는 회한의 기회를 주세요 ~ 그래야 문재인정권의 상실된 공정,정의,도덕,양심에 의해 살기힘들었던 국민들의 분노와 고통이 치유되리라 봅니다 ~ 총장의 사후거취가 정해지기전 이른바 헌법에도전한 물밑세력의 윈동력을 끊어버려야한다 미꾸라지들이 차후정국에 더는'

# 만든 함수를 이용해 textrank

In [26]:
# 키워드 추출
idx_to_vocab, vocab_to_idx = scan_vocabulary( sentences, tokenize= lambda x : komoran_tokenize(x) )
# 결과값: idx_to_vocab, vocab_to_idx
# vocab_to_idx

tokenizer = lambda x : komoran_tokenize(x)

In [27]:
# 단어간 유사도 측정
cooccurrence(tokens=idx_to_vocab, vocab_to_idx= vocab_to_idx)

<401x401 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [28]:
word_graph( sentences, tokenize=tokenizer )

(<401x401 sparse matrix of type '<class 'numpy.intc'>'
 	with 2954 stored elements in Compressed Sparse Row format>,
 ['하/VV',
  '것/NNB',
  '국민/NNG',
  '되/VV',
  '없/VA',
  '문재인/NNP',
  '수/NNB',
  '있/VV',
  '총장/NNP',
  '선거/NNG',
  '안철수/NNP',
  '윤석열/NNP',
  '서울/NNP',
  '만들/VV',
  '대통령/NNG',
  '윤/NNP',
  '정권/NNG',
  '정치/NNG',
  '시장/NNP',
  '보/VV',
  '이번/NNG',
  '시장/NNG',
  '받/VV',
  '때/NNG',
  '정권/NNP',
  '위하/VV',
  '말/NNG',
  '부산/NNP',
  '민주당/NNP',
  '시민/NNG',
  '생각/NNG',
  '당선/NNG',
  '단일/NNG',
  '같/VA',
  '후보/NNG',
  '야권/NNG',
  '총장/NNG',
  '검찰/NNG',
  '대한민국/NNP',
  '양보/NNG',
  '년/NNB',
  '개헌/NNP',
  '선/NNG',
  '자/NNB',
  '가/VV',
  '안/NNG',
  '알/VV',
  '이/NNP',
  '좌파/NNP',
  '오세훈/NNP',
  '그렇/VA',
  '가덕도/NNP',
  '국가/NNG',
  '대하/VV',
  '정부/NNG',
  '사퇴/NNG',
  '오거돈/NNP',
  '사람/NNG',
  '크/VA',
  '지지/NNG',
  '일/NNG',
  '이다/NNP',
  '정책/NNG',
  '오/VV',
  '힘/NNG',
  '거/NNB',
  '마음/NNG',
  '길/NNG',
  '빵/NNG',
  '후보/NNP',
  '을/NNG',
  '사람/NNP',
  '서/VV',
  '개/NNB',
  '바람직/XR',
  '가능/XR',
  '좋/VA

In [29]:
def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [30]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

keywords = textrank_keyword( sentences,tokenize=tokenizer,min_count=2, window=2, min_cooccurrence=2)

In [31]:
np.array(keywords)

array([['하/VV', '15.89265960499569'],
       ['것/NNB', '10.801954726790099'],
       ['국민/NNG', '7.436208980732152'],
       ['되/VV', '5.995189643280383'],
       ['있/VV', '4.491232599276133'],
       ['없/VA', '4.428881914233009'],
       ['문재인/NNP', '4.301855637077842'],
       ['수/NNB', '4.170894114794577'],
       ['선거/NNG', '3.9512312491374013'],
       ['총장/NNP', '3.80254607655521'],
       ['만들/VV', '3.528286447573875'],
       ['안철수/NNP', '3.370679421571066'],
       ['윤석열/NNP', '3.3388124505527'],
       ['정권/NNG', '3.3229838732274093'],
       ['서울/NNP', '3.3029874989051753'],
       ['대통령/NNG', '3.253999017199872'],
       ['정치/NNG', '3.2182493596820216'],
       ['시장/NNP', '2.9989245261935418'],
       ['윤/NNP', '2.7659537516265007'],
       ['보/VV', '2.741577517064143'],
       ['받/VV', '2.6091289078074476'],
       ['시장/NNG', '2.6005023770478792'],
       ['이번/NNG', '2.426098575621456'],
       ['정권/NNP', '2.4159301069414996'],
       ['때/NNG', '2.32985511818239'],
       

### python에서 textRank만들기

In [32]:
from collections import Counter

def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w: c for w, c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab: idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [33]:
idx_to_vocab, vocab_to_idx = scan_vocabulary( sentences, tokenize=tokenizer )
# vocab_to_idx # { '단어/품사' : 단어 인덱스 번호 }

In [34]:
from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

In [35]:
cooccurrence( sentences, vocab_to_idx=vocab_to_idx )

<401x401 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [36]:
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence)
    return g, idx_to_vocab

In [37]:
g, idx_to_vocab = word_graph( sentences, tokenize=tokenizer )

In [38]:
import networkx as nx 
nx.graph_atlas_g( ) 
# nx.pagerank(g, weight='coor_count')

 ...]

###  pytho에서 textRank 2

In [39]:
def scan_vocabulary(sentences, min_count=1, stop_words=set()):
    """
    sentence로부터 vocab를 min_count를 고려하여 vocab를 생성
    * pos를 고려하여, noun, verb, 형용사 만 남기고 다 날려버리는 것이 필요함.
    """
    word_lst = (s.split(" ") for s in sentences) # o
    # 결과: word_lst # [], [], [], [], [], ...
    
    word_lst = ([w for w in w_l if w not in stop_words] for w_l in word_lst) # o
    # 결과: ( [''], [''], [''], ... )
    
    word_counter = Counter(chain.from_iterable(word_lst))
    word_counter = {w: c for w, c in word_counter.items() if c >= min_count}
    # idx_to_vocab: count가 높을 수록 앞에 등장
    idx_to_vocab = [w for w, c in sorted(
        word_counter.items(), key=lambda x: -x[1])]
    vocab_to_idx = {w: idx for idx, w in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [40]:
# 한국어 불용어 사전 불러오기
stop_words = pd.read_excel('data/한국어 불용어 사전.xlsx', header=None)
stop_words = stop_words[0].to_list()

In [41]:
word_lst = (s.split(" ") for s in sentences) # o
# 결과: word_lst # [], [], [], [], [], ...

word_lst = ([w for w in w_l if w not in stop_words] for w_l in word_lst) # o
word_lst

<generator object <genexpr> at 0x000001B9318D02C8>